# Modelo de LSTM para previsão do ibovespa

## Bibliotecas

In [100]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from pathlib import Path
import os
pd.options.mode.chained_assignment = None

## Ler base

In [101]:
# Caminho da base
base_diretório = Path(os.getcwd())
diretório_princapl = base_diretório.parent
caminho = diretório_princapl / 'Base' / 'Base Ibovespa Tratada.csv'

# Carrega a base
acao = pd.read_csv(caminho)
acao['Data'] = pd.to_datetime(acao['Data'])

# Ordena por data (caso não esteja)
acao = acao.sort_values(by="Data")

acao

,Data,Último,Abertura,Máxima,Mínima,Variacao (%)
0,2015-01-05,47517.0,48512.0,48512.0,47264.0,-0.0205
1,2015-01-06,48001.0,47517.0,48061.0,47338.0,0.0102
2,2015-01-07,49463.0,48006.0,49882.0,48006.0,0.0305
3,2015-01-08,49943.0,49463.0,50261.0,49017.0,0.0097
4,2015-01-09,48840.0,49955.0,49955.0,48501.0,-0.0221
...,...,...,...,...,...,...
2577,2025-05-28,138888.0,139541.0,139547.0,138580.0,-0.0047
2578,2025-05-29,138534.0,138869.0,139108.0,137993.0,-0.0025
2579,2025-05-30,137027.0,138546.0,138637.0,136726.0,-0.0109
2580,2025-06-02,136787.0,137026.0,138471.0,136483.0,-0.0017


## Aplicando modelo de ML

In [102]:
# Define ontem e anteontem
ontem = acao['Data'].max()
anteontem = acao['Data'].iloc[-2]

print(f'Ontem: {ontem}\nAnteontem: {anteontem}')

Ontem: 2025-06-03 00:00:00
Anteontem: 2025-06-02 00:00:00


In [103]:
# Filtrando da base de treino até anteontem
acao_treino = acao[acao['Data'] < ontem]
cotacao_treino = acao_treino['Último'].to_numpy().reshape(-1, 1)

In [104]:
# Armazenar tamanho dos dados de treinamento
tamanho_dados_treinamento = int(len(cotacao_treino) * 1)
tamanho_dados_treinamento

2581

In [105]:
# Escala os dados
escalador = MinMaxScaler(feature_range=(0, 1))
dados_escalados = escalador.fit_transform(cotacao_treino)

In [106]:
# Cria dados de entrada para o modelo
treinamento_x = []
treinamento_y = []

for i in range(60, len(dados_escalados)):
    treinamento_x.append(dados_escalados[i-60:i, 0])
    treinamento_y.append(dados_escalados[i, 0])

In [107]:
treinamento_x, treinamento_y = np.array(treinamento_x), np.array(treinamento_y)
treinamento_x = treinamento_x.reshape(treinamento_x.shape[0], treinamento_x.shape[1], 1)

In [108]:
# Define e treina o modelo
modelo = Sequential()
modelo.add(LSTM(100, return_sequences=True, input_shape=(treinamento_x.shape[1], 1)))
modelo.add(LSTM(50, return_sequences=False))
modelo.add(Dense(25))
modelo.add(Dense(1))
modelo.compile(optimizer="adam", loss="mean_squared_error")
modelo.fit(treinamento_x, treinamento_y, batch_size=10, epochs=20)

Epoch 1/20


c:\Users\YUPOPIC\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


253/253 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - loss: 0.0268
Epoch 2/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.0010
Epoch 3/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 6.7215e-04
Epoch 4/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 6.5975e-04
Epoch 5/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 5.8633e-04
Epoch 6/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 7.1755e-04
Epoch 7/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 4.3082e-04
Epoch 8/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 4.2711e-04
Epoch 9/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 3.9014e-04
Epoch 10/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 3.9404e-04
Epoch 11/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 2.8918e-04
Epoch 12/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 2.5234e-04
Epoch 13/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 3.5421e-04
Epoch 14/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 2.8740e-04
Epoch 

In [109]:
# Pega os últimos 60 dias até ANTEONTEM para previsão
ultimos_60 = dados_escalados[-120:]
entrada_predicao = ultimos_60.reshape(1, 120, 1)

In [113]:
predicao = modelo.predict(entrada_predicao)
predicao = escalador.inverse_transform(predicao)

# Valor real de ontem
valor_real = acao[acao['Data'] == ontem]['Último'].values[0]
valor_anterior = acao[acao['Data'] == anteontem]['Último'].values[0]

# Calcular a variação percentual
variacao_real = (valor_real - valor_anterior) / valor_anterior * 100
variacao_predita = (predicao[0][0] - valor_anterior) / valor_anterior * 100

# Direção correta?
direcao_correta = (variacao_real * variacao_predita) > 0  # ambos com mesmo sinal

# Tabela final
df_previsao = pd.DataFrame([{
    'Data': ontem.date(),
    'Valor Real': f"{valor_real:.2f}",
    'Valor Previsto': f"{predicao[0][0]:.2f}",
    'Variação Real (%)': f"{variacao_real:.3f}",
    'Variação Prevista (%)': f"{variacao_predita:.3f}",
    'Direção Correta?': 'Sim' if direcao_correta else 'Não'
}])

colunas_para_converter = ['Valor Real', 'Valor Previsto', 'Variação Real (%)', 'Variação Prevista (%)']

for coluna in colunas_para_converter:
    df_previsao[coluna] = df_previsao[coluna].astype(float)

df_previsao

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


,Data,Valor Real,Valor Previsto,Variação Real (%),Variação Prevista (%),Direção Correta?
0,2025-06-03,136685.0,136688.45,-0.075,-0.072,Sim


# Métricas de avaliação do nosso modelo de ML

### Mean Absolute Error (MAE) e Mean Absolute Percentage Error (MAPE)

In [118]:
# A média das diferenças absolutas entre as previsões e os valores reais. Penaliza todos os erros da mesma forma
mae = mean_absolute_error(df_previsao['Valor Real'], df_previsao['Valor Previsto'])
print(f"Mean Absolute Error (MAE): {mae:.2f}")

# Calcular o MAPE
mape = mean_absolute_percentage_error(df_previsao['Valor Real'], df_previsao['Valor Previsto'])
print(f"Mean Absolute Percentage Error (MAPE): {mape:.4f} = {100*mape:.4f}%")

Mean Absolute Error (MAE): 3.45
Mean Absolute Percentage Error (MAPE): 0.0000 = 0.0025%


### Root Mean Squared Error (RMSE)

In [119]:
# A raiz quadrada da média dos erros ao quadrado. Dá mais peso a grandes erros
rmse = np.sqrt(mean_squared_error(df_previsao['Valor Real'], df_previsao['Valor Previsto']))
mean_price = df_previsao['Valor Real'].mean()

print(f"RMSE: {rmse:.2f}")
print(f"RMSE percentual: {(rmse / mean_price):.4f} = {100*(rmse / mean_price):.3}%")

RMSE: 3.45
RMSE percentual: 0.0000 = 0.00252%
